In [1]:
import ee
# ee.Authenticate()
ee.Initialize()

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import JSON
import altair as alt

In [10]:
# Functions to rename Landsats
#landsat 1-2-3-4M-5M have only 6 bands 
def renameL1_5M(img):
    return img.rename(['GREEN', 'RED', 'NIR', 'NIR2','QA_PIXEL','QA_RADSAT'])

# landsat 4T-5T-7 have similar bands with the following meanings 
def renameL457(img):
    return img.rename(['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1',
        'SWIR2', 'TEMP1', 'ATMOS_OPACITY', 'QA_CLOUD','ATRAN', 'CDIST','DRAD', 
        'EMIS', 'EMSD', 'QA', 'TRAD', 'URAD','QA_PIXEL','QA_RADSAT'])

# landsat 8 has an extra band in 0 place, so the first one is not blue
def renameL8(img):
    return img.rename(['AEROS', 'BLUE', 'GREEN', 'RED', 'NIR',
        'SWIR1','SWIR2', 'TEMP1', 'QA_AEROSOL', 'ATRAN', 'CDIST','DRAD', 'EMIS',
        'EMSD', 'QA', 'TRAD', 'URAD', 'QA_PIXEL', 'QA_RADSAT'])



In [19]:
# call and rename all the Landsat collections
l1 = ee.ImageCollection("LANDSAT/LM01/C02/T1").map(renameL1_5M)
l2 = ee.ImageCollection("LANDSAT/LM02/C02/T1").map(renameL1_5M)
l3 = ee.ImageCollection("LANDSAT/LM03/C02/T1").map(renameL1_5M)
lM4 = ee.ImageCollection("LANDSAT/LM04/C02/T1").map(renameL1_5M)
lM5 = ee.ImageCollection("LANDSAT/LM05/C02/T1").map(renameL1_5M)
lT4 = ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').map(renameL457)
lT5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').map(renameL457)
l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').map(renameL457)
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').map(renameL8)

In [20]:
# Functions to add masks based on QA_PIXEL bit info
def addMask(img):
    """
    img.select('QA_PIXEL').bitwiseAnd(64)
    this piece checks wether the 6th bit of QA_PIXEL is 1 or not. from doc we have
    Bit 6: Clear
        0: Cloud or Dilated Cloud bits are set
        1: Cloud and Dilated Cloud bits are not set
    so want to be 1
    """
    clear = img.select('QA_PIXEL').bitwiseAnd(64).neq(0)
    clear = clear.updateMask(clear).rename(['pxqa_clear'])

    water = img.select('QA_PIXEL').bitwiseAnd(128).neq(0)
    water = water.updateMask(water).rename(['pxqa_water'])

    cloud_shadow = img.select('QA_PIXEL').bitwiseAnd(16).neq(0)
    cloud_shadow = cloud_shadow.updateMask(cloud_shadow).rename(['pxqa_cloudshadow'])

    snow = img.select('QA_PIXEL').bitwiseAnd(32).neq(0)
    snow = snow.updateMask(snow).rename(['pxqa_snow'])

    masks = ee.Image.cat([clear, water, cloud_shadow, snow])

    return img.addBands(masks)

In [ ]:
# masking out clouds function 
def maskQAClear(img):
    return img.updateMask(img.select('pxqa_clear'))

In [21]:
#Function to add GCVI as a band.
def add_NDVI(img):
    
    NDVI = img.normalizedDifference(['NIR', 'RED']).rename('NDVI')
  
    return ee.Image.cat([img, NDVI])


In [23]:
# Merge Landsats
landsat = ee.ImageCollection(l1.merge(l2).merge(l3).merge(lM4).merge(lM5).merge(lT4).merge(lT5).merge(l7).merge(l8)).sort('system:time_start')
landsat = landsat.map(addMask).map(maskQAClear).map(add_NDVI)

In [87]:
long = -101.8573594
lat = 43.9498017
point = ee.Geometry.Point([long,lat])

In [88]:
scale = 30
res = landsat.select('NDVI').getRegion(point,scale)

In [89]:
res = res.getInfo()

In [90]:
df = pd.DataFrame(res)
headers = df.iloc[0]
df = pd.DataFrame(df.values[1:], columns=headers)

In [91]:
def add_date_info(df):
    df['Timestamp'] = pd.to_datetime(df['time'], unit='ms')
    df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
    df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
    df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
    df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
    return df

In [92]:
add_date_info(df)

,id,longitude,latitude,time,NDVI,Timestamp,Year,Month,Day,DOY
0,1_1_1_1_1_1_2_LM03_034029_19780928,-101.85723,43.949851,275849434091,None,1978-09-28 16:50:34.091,1978,9,28,271
1,1_1_1_2_LT04_033029_19821116,-101.85723,43.949851,406314255921,0.053389,1982-11-16 17:04:15.921,1982,11,16,320
2,1_1_1_2_LT04_032029_19821125,-101.85723,43.949851,407091502569,None,1982-11-25 16:58:22.569,1982,11,25,329
3,1_1_1_2_LT04_032030_19821125,-101.85723,43.949851,407091526679,None,1982-11-25 16:58:46.679,1982,11,25,329
4,1_1_1_2_LT04_033029_19821218,-101.85723,43.949851,409079062274,None,1982-12-18 17:04:22.274,1982,12,18,352
...,...,...,...,...,...,...,...,...,...,...
2808,2_LC08_032030_20221201,-101.85723,43.949851,1669915853235,None,2022-12-01 17:30:53.235,2022,12,1,335
2809,2_LC08_033029_20221208,-101.85723,43.949851,1670520998923,None,2022-12-08 17:36:38.923,2022,12,8,342
2810,2_LC08_032029_20221217,-101.85723,43.949851,1671298222625,-0.040591,2022-12-17 17:30:22.625,2022,12,17,351
2811,2_LC08_033029_20230109,-101.85723,43.949851,1673285789386,0.076755,2023-01-09 17:36:29.386,2023,1,9,9


In [94]:
df_one_year = df[['NDVI','DOY']].groupby(['DOY']).mean()

In [96]:
df_one_year['doy'] = np.arange(1,367)

In [93]:
alt.Chart(df).mark_bar(size = 1.5,color='green').encode(
    alt.X('Timestamp:T'),
    alt.Y('NDVI:Q'),
    color=alt.Color(
    'DOY:Q', scale=alt.Scale(scheme='redblue', domain=(1, 300))),
    tooltip=[
        alt.Tooltip('DOY:Q', title='doy'),
        alt.Tooltip('Month:Q', title='Month')
    ]
).properties(width=4000, height=400)

alt.Chart(...)

In [100]:
df_one_year

,NDVI,doy
DOY,,
1,0.037511,1
2,0.057540,2
3,0.082266,3
4,0.031283,4
5,0.042321,5
...,...,...
362,NaN,362
363,NaN,363
364,0.019940,364


In [106]:
alt.Chart(df_one_year).mark_bar(size = 2).encode(
    alt.X('doy:Q'),
    alt.Y('NDVI:Q'),
).properties(width=1000, height=400)

alt.Chart(...)